# Incelnator 3000 - Gerador de roteiros cinematográficos


# Introdução
O Incelnator 3000 surgiu a partir da observação de um fenômeno emergente (que também podemos chamar de "meme") da Internet, que é um grupo de pessoas que se autodenominam – ou não – de *Incel*. Os filmes de incel se tornaram recentemente uma "piada interna" na web e por isso escolhemos minunciosamente os roteiros de algumas dessas obras cinematográficas para afinar um modelo GPT-2, para que seja possível reproduzir mais dessas pérolas do cinema.

## Afinal, o que é um Incel?
Incel é uma sigla para *involuntary celibatary* (celibatário involuntário). Estas pessoas se denominam parte desse grupo aqueles que não conseguem, por culpa deles ou não, se relacionar romanticamente com nenhuma mulher. Como a maioria dos integrantes são realmente pessoas insuportáveis, rapidamente os incels se tornaram uma "facção" **odiada** na Internet. Há vários *memes* sobre incels que podem ser encontrados ao navegar pela web.

## O que define um "filme de incel"
Agora que sabemos o que é um incel, é possível analisar que um bocado dos filmes escolhidos para este projeto não possuem personagens, protagonistas ou não, que se encaixem na definição de incel. Isto se dá porque algumas dessas obras cinematográficas são **as favoritas deste grupo**, já que muitos dos rapazes incels se espelham ou acham que são parecidos com os protagonistas desses filmes.

## Os filmes utilizados no treinamento
*   (500) Dias com Ela (2009)
*   Batman: O Cavaleiro das Trevas (2008)
*   Blade Runner - O Caçador de Androides (1982)
*   Cães de Aluguel (1992)
*   Clube da Luta (1999)
*   Coringa (2019)
*   Donnie Darko (2001)
*   Drive (2011)
*   Ela (2013)
*   O Poderoso Chefão (1972)
*   O Rei da Comédia (1983)
*   Psicopata Americano (2000)
*   Pulp Fiction: Tempo de Violência (1994)
*   Scott Pilgrim contra o Mundo (2010)
*   Taxi Driver – Motorista de Táxi (1976)

## Os filmes que não conseguimos arranjar o roteiro, mas que deveriam estar aqui
*   Blade Runner 2049 (2017)
*   Matrix (1999)
*   O Batman (2022)
*   O Farol (2019)
*   O Abutre (2014)
*   Scarface (1983)

## Projeto que serviu de inspiração – e grande ajuda
https://towardsdatascience.com/film-script-generation-with-gpt-2-58601b00d371

# Código

## Preparação do ambiente

Instalando o módulo *transformers* da HuggingFace, que nos possibilita usar transformadores, um tipo de rede neural, e, consequentemente, utilizar um modelo pré-treinado do Distil GPT-2.

Link de vídeo falando sobre transformadores - https://www.youtube.com/watch?v=SZorAJ4I-sA

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 49.1 MB/s 
     |████████████████████████████████| 1.1 MB 5.5 MB/s 
     |████████████████████████████████| 212 kB 53.3 MB/s 
     |████████████████████████████████| 127 kB 47.9 MB/s 
     |████████████████████████████████| 144 kB 47.1 MB/s 
     |████████████████████████████████| 271 kB 44.3 MB/s 
     |████████████

Importando os módulos que serão utilizados e conectando o notebook ao Google Drive.

In [ ]:
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup, WEIGHTS_NAME, CONFIG_NAME, pipeline

import pickle
import logging
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Definindo onde será treinado/afinado o modelo.

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
device

'cpu'

Variáveis de caminhos de arquivos para serem utilizadas mais tarde.

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Incelnator3000/models"

In [ ]:
FILE_PATH = os.path.join("/content/drive/MyDrive/Incelnator3000", "incel_scripts.txt")

Aqui retiramos um pedaço do código, que é a classe para os scripts, do artigo supracitado. No método "__init__" será executada a fase de tokenização, que é de suma importância para o uso de transformadores, que é um de seus pontos principais, chamado de Encoding Posicional.

In [ ]:
class ScriptData(Dataset):
    def __init__(self, tokenizer, file_path: str, block_size = 256, overwrite_cache = False,):
        assert os.path.isfile(file_path) # Essa linha verifica se o pathfile é realmente um arquivo, e não um diretório ou qualquer coisa do tipo

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence) # Separa-se pedaços do arquivo em blocos menores para ser mais facilmente processado

        directory, filename = os.path.split(file_path)

        cached_features_file = os.path.join(directory, "gpt2" + "_" + str(block_size) + "_" + filename) # Variável do nome do arquivo com os textos divididos em blocos

        if os.path.exists(cached_features_file) and not overwrite_cache: # Processo de serialização dos dados utilizando o módulo Pickle, caso exista o arquivo, ele lê
            logging.info(f"Loading features from your cached file {cached_features_file}")
            with open(cached_features_file, "rb") as cache:
                self.examples = pickle.load(cache)
                logging.debug("Loaded examples from cache")
        else:                                                            # caso o arquivo não exista, ele cria
            logging.info(f"Creating features from file {filename} at {directory}")

            self.examples = []
            with open(file_path, encoding = "utf-8") as f: # Armazena o texto do arquivo em uma variável chamada "text"
                text = f.read()
                logging.debug("Succesfully read text from file")

            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))  # Tokeniza o texto e armazena as IDs dos tokens em uma variável chamada "tokenized_text"

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):
                self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size]))  # Armazena cada input construído a partir de cada bloco em uma lista chamada "examples"

            logging.info(f"Saving features into cached file {cached_features_file}") # Salva os inputs construídos no arquivo criado pelo método "__init__"
            with open(cached_features_file, "wb") as cache:
                pickle.dump(self.examples, cache, protocol = pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype = torch.long)

## Pré-Treino

Instalamos o Distil GPT-2, pois não tínhamos muita liberdade de escolher modelos mais pesados, completos e robustos. Chegamos até a tentar usar o GPT-2 Medium, mas acabou ficando muito pesado pro Google Colab rodar.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token = '<|startoftext|>')
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

model = GPT2LMHeadModel.from_pretrained('distilgpt2', bos_token_id = tokenizer.bos_token_id)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Registrando IDs de tokens importantes para utilizar na hora do treinamento.

In [ ]:
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id

50256

Criando a variável que irá receber a classe ScriptData, então é neste momento em que está ocorrendo a tokenização dos roteiros.

In [ ]:
dataset = ScriptData(tokenizer = tokenizer, file_path = FILE_PATH )

Para facilitar a iteração dos itens do dataset, utilizamos a função DataLoader e a armazenamos na variável "script_loader" que será utilizada durante o treinamento. Aqui também dividimos os batches de cada iteração.

In [ ]:
script_loader = DataLoader(dataset, batch_size = 2, shuffle = True)

Aqui temos as variáveis de treinamento. Optamos por treinar por três épocas, pois não queríamos estourar a RAM do Colab, problema que acarretou o atraso da entrega do projeto. 

In [ ]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 0.00002  # Retirado do código do artigo, utilizamos o 
WARMUP_STEPS = 10000     # mesmo learning rate e warm up steps

## Afinação/Treinamento do Modelo

Ajustando o modelo com otimizador e um agendador.

In [ ]:
model = model.to(device) # Garantindo que o modelo está rodando no dispositivo
model.train() # Ativando o modo treino do GPT-2

optimizer = AdamW(model.parameters(), lr = LEARNING_RATE) # No artigo é utilizado o algoritmo adaptativo de gradiente chamado Adam, para otimização do treino 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = WARMUP_STEPS, num_training_steps = -1) # Aplica o otimizador em um esquema que irá dar uma "aquecida" na taxa de aprendizado
                                                                                                                 # e depois fará ela descrescer até 0

Variáveis para serem utilizadas durante o treino.

In [ ]:
script_count = 0
sum_loss = 0.0
batch_count = 0

Célula na qual é feita o treinamento e há impressões de outputs durante a afinação do modelo. Este pedaço do código também foi extraído do artigo e pouco alterado.

In [ ]:
for epoch in range(EPOCHS):
    print(f"EPOCH {epoch} started" + '=' * 30)
    for idx, script in enumerate(script_loader):
        outputs = model(script.to(device), labels=script.to(device)) # Salva-se os outputs do tensor tokenizado em "outputs"
        
        loss, logits = outputs[:2] # Recupera-se só os dois primeiros outputs (loss e logits) para utilizar para o backward pass
        loss.backward()
        sum_loss = sum_loss + loss.detach().data # Registra o loss para observação durante o treino
                       
        script_count = script_count + 1
        if script_count == BATCH_SIZE:  # Passos a serem tomados para o otimizador e o agendador
            script_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 200: # Caso chegue ao batch 200, imprime-se uma geração do modelo sem necessidade de input, para fins de visualização do treino
            model.eval() # Coloca-se o modelo em modo avaliatório
            print(f"sum loss {sum_loss}") # Imprime-se o somatório da loss, para ter um relatório do treino
            sample_outputs = model.generate(
                                    pad_token_id = eos_id,
                                    eos_token_id = eos_id,
                                    bos_token_id = bos_id,
                                    do_sample = True,   
                                    repetition_penalty = 1.1, # Colocamos repetition penalty nessa etapa só para verificar como ele estava gerando sem a formatação do roteiro
                                    top_k = 50, 
                                    min_length = 200,
                                    max_length = 1000,
                                    top_p = 0.95, 
                                    num_return_sequences = 1
                              )

            print("Output:\n" + 100 * '-')
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            batch_count = 0
            sum_loss = 0.0
            model.train()  # Após gerar o texto no batch 200, resetamos o batch_count e o sum_loss e voltamos a treinar o modelo

EPOCH 0 started==============================
sum loss 258.07452392578125
Output:
----------------------------------------------------------------------------------------------------
0: ?

  Then the car pulls out, and Mr. Gaff looks at it again: The man in uniform is lying unconscious...
CONTINUED (O.S.) - MORNING 12/24 6 AM COMMENTARY 7 PGS./ $4 K / 9PK MS., SALE MAY 6 11AM EST -- NIGHT 13 AESTILES 10A WEST #1 IN THE MACHINE. In his bed there's an image of him with a clown figure that hangs around by some distance from its mouth like this..MATHLETICS MAN on T-shirt LABIDERSHIP STREET. He sits close to them; WELL ANGLE through VIEWING. HOMICULTURISTIAN EXPLOSIVES. Joker stands upright next TO HEAR THAT GUY. JOKER UNDERSTAGE ON HIS FACE... BITCH! This one is REAL TOM!!! Look up!! There are TWO BOYS playing with each other all over CAMERA. It IS THE FIRST BOY!!! (Caring) CLICK BELOW To SEE IT LIVE! Now come closer down below Tom Batty's face when he KNOWS HIM HERE. And then the OTHER VO

## Registro do Modelo

In [ ]:
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model.state_dict(), output_model_file)
model.config.to_json_file(output_config_file)

tokenizer.save_vocabulary(OUTPUT_DIR)

('/content/drive/MyDrive/Incelnator3000/models/vocab.json',
 '/content/drive/MyDrive/Incelnator3000/models/merges.txt')

## Geração de Roteiros de Filme Incel

Recuperamos o modelo e o tokenizador que estão salvos no diretório previamente registrado na variável "OUTPUT_DIR"

In [ ]:
model_trained = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model_trained = model_trained.to(device)
tokenizer_trained = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)

Colocamos o modelo em modo de avaliação/geração

In [ ]:
model_trained.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

Utilizamos o método *pipeline* para a geração de outputs, pois estávamos tendo problemas com o "model.generate()". Todos os métodos são do módulo *transformers* da HuggingFace.

In [ ]:
revisor = pipeline('text-generation', tokenizer = tokenizer_trained, model = model_trained)

In [ ]:
print(revisor("TYLER", # O método funciona melhor usando um input id, decidi colocar "TYLER" em maiúsculo, pois iniciaria uma fala de Tyler, de acordo com as regras dos roteiros cinematográficos
              handle_long_generation = 'hole', 
              top_k = 500,
              pad_token_id=50256,
              eos_token_id=50256,
              bos_token_id=50257,
              min_length = 200,
              max_length = 1000,
              top_p=0.95)[0].get('generated_text'))

TYLER
                                                                                    93.


LIVING TIES WILL CABLE FOR 11 AM, the water has SPOKED.

                                                    BERNIE ROSE
                                  55.

                               SCOTT
                               (silence)
                             You've been afraid too.  I tell you.

                                                         72.


64    EXT. LANACOLI - DAY                            61

            

                   

                

                 

                  

                

              

           

              

             

            

           

           

           

            

            

            

            

            

           

             

            

              

             

               

           

              

             

              

           

              

       